In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models
from astropy import units as u
from astropy.constants import c,h
from astropy.io import fits
import pandas as pd
from scipy import integrate

#Convert c and h to cgs
c = c.cgs
h = h.cgs

# 1.)

## Source Object

In [ ]:
class Source:

    def __init__(self,name,mag):
        """Instantiation of the Source.

        This function initializes the Source class and all parameters concerning it

        Ex:
            Source("Vega")

        Args:
            None

        """
        self.name = name
        self.mag = mag

        #Need to set object's SED and magnitude here. Not sure how.



    def photon_flux(self,type, T, lam):
        """ Determines photon flux at given wavelength of the source

        Takes type (e.g. blackbody), temperature (if blackbody), and wavelength and
        returns flux

        Parameters
        ----------
        type: String for source type ("blackbody" or "###")
        T: Temperature in K for use ONLY when type is "blackbody"
        lam: wavelength in angstroms

        Returns
        -------
        f_photon: Photon flux
        """

        #Attach units of angstroms to lam
        lam = lam * u.AA
        nu = c/lam

        if(type == "Blackbody"):
            F = models.BlackBody(T*u.K)
            f_lam = F(lam)*(nu**2)/(c*1e8)
            f_lam = f_lam*(u.erg**-1)*(u.AA**2)*u.cm*u.hertz*u.sr*(u.s**2)
        else:
            f_lam = 0

        #Convert to photon flux
        f_photon = (f_lam*lam/(h*c))*10**(-0.4*self.mag)

        return(f_photon)

# 2.)

## Atmosphere Object

In [ ]:
class Atmosphere:

    def __init__(self,name):
        """Instantiation of the Atmosphere.

        This function initializes the Atmosphere class and all parameters concerning it

        Args:
            None

        """
        self.name = name
        self.a_lam = 0.8

    def calcSky(self,fileName,filtLow,filtHigh,area,plateScale):
        """ Calculate sky background level.

        This function takes a sky spectrum fits file name and filter bounds
        and returns the sky background flux

        Parameters
        ----------
        fileName: fits file in same dir as script
        area: telescope area
        plateScale: plateScale of detector (arcsec/pix)

        Returns
        -------
        fSky: sky flux
        """

        #Import fits file containing sky spectrum
        sky = fits.open(fileName)
        sky_array = pd.DataFrame(sky[1].data)
        lamTot = sky_array['lam'] * 10 #Convert to AA
        fluxTot = sky_array['flux'] * 1e-4 #convert to 1/AA

        #Remove wavelengths and fluxes outside of filter
        lam = lamTot[(lamTot<filtHigh) == True]
        flux = fluxTot[(lamTot<filtHigh) == True]
        flux = flux[(lam>filtLow) == True]
        lam = lam[(lam>filtLow) == True]

        #Change flux from photon/s/m^2/um/arcsec to photon/s/um/pix
        flux = flux*(area*1e-4)*(plateScale**2)

        #Compute the total flux
        fSky = integrate.simps(y=flux,x=lam)

        return(fSky)



# 3.)

## Telescope Object

In [ ]:
class Telescope:

    def __init__(self,name,diameter):
        """Instantiation of the Telescope.

        This function initializes the Telescope class and all parameters concerning it

        Ex:
            Source("TMO")

        Args:
            name: string
            diameter: objective diameter in cm

        """

        self.name = name
        self.diameter = diameter
        self.area = (np.pi/4)*diameter**2
        self.m_lam = 0.5**(1/3)


########################### DETECTOR CLASS #####################################

class Detector:

    def __init__(self,name,plateScale,sig_rn):
        """Instantiation of the Detector.

        This function initializes the Detector class and all parameters concerning it

        Ex:
            Source("CCD")

        Args:
            None

        """
        self.name = name
        self.d_lam = 0.5**(1/3)

        #Set plate scale
        self.plateScale = plateScale #arcsec/pix TBD

        #Set readout noise
        self.sig_rn = sig_rn

# 4.)

## Instrument Object

In [ ]:
class Instrument:

    def __init__(self,name):
        """Instantiation of the Instrument.

        This function initializes the Instrument class and all parameters concerning it

        Ex:
            Source("TMO")

        Args:
            None

        """
        self.name = name
        self.i_lam = 0.5**(1/3)

# 5.)

## Filter Object

In [ ]:
class Filter:

    def __init__(self,name):
        """Instantiation of the Filter.

        This function initializes the Filter class and all parameters concerning it

        Ex:
            Source("V")

        Args:
            None

        """
        self.name = name

        #Define lower and upper wavelength bounds
        if(name=="V"):
            self.lower = 5500-850/2
            self.higher = 5500+850/2

    def response(self, band, lam):
        """ Determines filter response

        Takes wavelength and returns filter response

        Parameters
        ----------
        lam:  wavelength in angstroms

        Returns
        -------
        f_lam: filter response
        """

        i_lam = 0


        if(lam>=self.lower and lam<=self.higher):
            i_lam = 0.8


        return(i_lam)

# 6.)

## Signal Calculation

In [ ]:
#Create range of wavelengths
lam = np.arange(3000,7500,1)

#Initialize the classes
src = Source("Vega",30)
atm = Atmosphere("earth")
tel = Telescope("TMO", 30)
ins = Instrument("TMO")
det = Detector("CCD",0.5,1)
fil = Filter("V")

#Integrate over wavelengths
F_lam = src.photon_flux("Blackbody", 6000 ,lam)
a_lam = atm.a_lam
m_lam = tel.m_lam
i_lam = ins.i_lam

f_lam = []
for i in range(len(lam)):
    f_lam.append(fil.response("V",lam[i]))

d_lam = det.d_lam
T = tel.area

Sprime = F_lam*a_lam*m_lam*i_lam*f_lam*d_lam
S = integrate.simps(y=Sprime,x=lam)

print("The signal for: \n",
        "\t-A Star of magnitude 30\n",
        "\t-SED that is a blackbody of T = 6000 K\n",
        "\t-Telescope of diamater 30 cm\n",
        "\t-In the V-band\n",
        "\t-With constant atmosphere, instrumental, and mirror transmissions\n",
        "\t-With constant detector response\n",
        "= {} photons/s/cm^2\n".format(S))

The signal for: 
 	-A Star of magnitude 30
 	-SED that is a blackbody of T = 6000 K
 	-Telescope of diamater 30 cm
 	-In the V-band
 	-With constant atmosphere, instrumental, and mirror transmissions
 	-With constant detector response
 = 2.300522738839605 photons/s/cm^2



# Exposure Calculator II

In [ ]:
class Signal:

    def __init__(self,name,T,F,sig_rn,Npix,B):
        """Instantiation of the Signal being collected.

        This function initializes the Signal class and all parameters concerning it

        Args:
          name: name for signal
          T: telescope area
          F: photon flux either over bandpass or at wavelength
          a_lam: atmospheric transmission per wavelength
          m_lam: mirror throughput per wavelength
          i_lam: instrumental throughput per wavelength
          f_lam: filter throughput per wavlength
          d_lam: detector response per wavlength
          sig_rn: readout noise per pixel
          Npix: number of pixels from aperture size
          B: sky background signal (photon/s/pix)

        """
        self.name = name
        self.T = T
        self.F = F
        self.sig_rn = sig_rn
        self.Npix = Npix
        self.B = B


    def SNR(self,t):
        """ Determines signal to noise ratio

        Takes type of detector, throughput/transmission terms, wavelenghts, and
        exposure time and returns SNR

        Parameters
        ----------
        t: exposure time

        Returns
        -------
        SNR: signal to noise ratio (either single value or array)
        """

        #Compute SNR
        SNR = self.F*self.T*t/np.sqrt(self.F*self.T*t + self.B*self.T*t*self.Npix + self.Npix*self.sig_rn**2)


        return(SNR)

    def exp_time(self,SNR):
        """ Determines exposure time ratio

        Takes type of detector, throughput/transmission terms, wavelengths, and
        SNR and returns exposure time

        Parameters
        ----------
        lam:  wavelength in angstroms
        SNR: signal to noise ratio (either single value or array)

        Returns
        -------
        t: exposure time
        """

        #Solve for signal using quadratic formula
        a = (self.F*SNR)**-2
        b = -(self.F + self.B*self.Npix)
        c = -self.Npix*self.sig_rn**2
        t = ((-b + np.sqrt(b**2 - 4*a*c))/(2*a))/self.T #Always take positive value

        return(t)


# Exposure Calculator III

Test sky background calculation and SNR/Exposure Time calculation

In [ ]:
#Test calcSky
B = atm.calcSky('sky_0.0.fits',fil.lower,fil.higher,tel.area,det.plateScale)

sig = Signal('test',tel.area,S,det.sig_rn,30,B)

#Calculate SNR for 10 sec exposure
SNR = sig.SNR(10)

#Calculate exposure time for SNR = 50
t = sig.exp_time(50)

In [ ]:
print("The SNR for a 10 second exposure is: {}\n".format(SNR))

print("The exposure time to achieve an SNR of 50 is: {} sec\n".format(t))

The SNR for a 10 second exposure is: 55.66157744639027

The exposure time to achieve an SNR of 50 is: 225.93795355328618 sec

